In [15]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Ajouter le path parent
sys.path.append('../')

# Imports essentiels
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import datetime

# Configuration Plotly
pio.renderers.default = "browser"  # Pour éviter les erreurs nbformat

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

print(f"🚀 Setup terminé - {datetime.now().strftime('%H:%M:%S')}")

🚀 Setup terminé - 23:37:26


In [16]:
def load_clean_data():
    """Charge les données nettoyées disponibles"""
    
    # Chemins possibles
    paths = [
        '../data/processed/games_clean_balanced_*.parquet',
        '../data/processed/games_clean_conservative_*.parquet', 
        '../data/processed/games_clean_*.parquet',
        '../data/processed/games_raw.parquet',
        '../data/processed/games_raw.csv'
    ]
    
    # Essayer de trouver un fichier
    import glob
    
    for path_pattern in paths:
        files = glob.glob(path_pattern)
        if files:
            latest_file = max(files, key=os.path.getctime)  # Plus récent
            print(f"📖 Chargement: {latest_file}")
            
            try:
                if latest_file.endswith('.parquet'):
                    return pd.read_parquet(latest_file)
                else:
                    return pd.read_csv(latest_file)
            except Exception as e:
                print(f"❌ Erreur chargement {latest_file}: {e}")
                continue
    
    # Fallback: données d'exemple
    print("🎯 Génération de données d'analyse d'exemple...")
    return generate_analysis_sample()

def generate_analysis_sample():
    """Génère des données d'exemple pour l'analyse"""
    np.random.seed(42)
    n_games = 2000
    
    # Ouvertures réalistes avec fréquences différentes
    openings = {
        'Sicilian Defense': 0.25,
        "Queen's Gambit": 0.15,
        'French Defense': 0.12,
        'Italian Game': 0.10,
        'English Opening': 0.08,
        'Caro-Kann Defense': 0.07,
        'Ruy Lopez': 0.06,
        'King\'s Indian Defense': 0.05,
        'Scandinavian Defense': 0.04,
        'Alekhine Defense': 0.03,
        'Nimzo-Indian Defense': 0.03,
        'Bird Opening': 0.02
    }
    
    # Générer les ouvertures selon leur popularité
    opening_list = []
    for opening, freq in openings.items():
        opening_list.extend([opening] * int(n_games * freq))
    
    # Compléter pour avoir exactement n_games
    while len(opening_list) < n_games:
        opening_list.append(np.random.choice(list(openings.keys())))
    
    opening_list = opening_list[:n_games]
    np.random.shuffle(opening_list)
    
    df = pd.DataFrame({
        'main_opening': opening_list,
        'white_elo': np.random.normal(1500, 300, n_games).astype(int),
        'black_elo': np.random.normal(1500, 300, n_games).astype(int),
        'moves_count': np.random.gamma(2.5, 12, n_games).astype(int) + 8,
        'time_control': np.random.choice(['Bullet', 'Blitz', 'Rapid', 'Classical'], 
                                       n_games, p=[0.4, 0.35, 0.2, 0.05]),
        'white_score': np.random.choice([0, 0.5, 1], n_games, p=[0.32, 0.28, 0.40])
    })
    
    # Colonnes dérivées
    df['avg_elo'] = (df['white_elo'] + df['black_elo']) / 2
    df['black_score'] = 1 - df['white_score']
    df['draw'] = (df['white_score'] == 0.5).astype(int)
    df['white_win'] = (df['white_score'] == 1.0).astype(int)
    df['black_win'] = (df['white_score'] == 0.0).astype(int)
    
    # Catégories ELO
    df['elo_category'] = pd.cut(df['avg_elo'], 
                               bins=[0, 1200, 1500, 1800, 2100, 3000],
                               labels=['Débutant', 'Intermédiaire', 'Avancé', 'Expert', 'Maître'])
    
    # Catégories de durée
    df['duration_category'] = pd.cut(df['moves_count'],
                                   bins=[0, 15, 30, 50, 200],
                                   labels=['Très courte', 'Courte', 'Moyenne', 'Longue'])
    
    print(f"✅ {len(df):,} parties d'exemple générées pour l'analyse")
    return df

# Charger les données
df = load_clean_data()
print(f"📊 {len(df):,} parties disponibles pour l'analyse")
print(f"📚 {df['main_opening'].nunique():,} ouvertures uniques")

📖 Chargement: ../data/processed\games_clean_conservative_20250607_160448.parquet
📊 49,360 parties disponibles pour l'analyse
📚 192 ouvertures uniques


In [17]:
class OpeningAnalyzer:
    """Analyseur complet des ouvertures d'échecs"""
    
    def __init__(self, df):
        self.df = df.copy()
        self.setup_analysis()
    
    def setup_analysis(self):
        """Prépare les données pour l'analyse"""
        
        # S'assurer que toutes les colonnes nécessaires existent
        required_cols = {
            'main_opening': 'Unknown',
            'white_elo': 1500,
            'black_elo': 1500,
            'white_score': 0.5,
            'moves_count': 40,
            'time_control': 'Blitz'
        }
        
        for col, default in required_cols.items():
            if col not in self.df.columns:
                self.df[col] = default
                print(f"⚠️ Colonne {col} manquante, valeur par défaut: {default}")
        
        # Colonnes dérivées
        if 'avg_elo' not in self.df.columns:
            self.df['avg_elo'] = (self.df['white_elo'] + self.df['black_elo']) / 2
        
        if 'black_score' not in self.df.columns:
            self.df['black_score'] = 1 - self.df['white_score']
        
        if 'draw' not in self.df.columns:
            self.df['draw'] = (self.df['white_score'] == 0.5).astype(int)
        
        if 'white_win' not in self.df.columns:
            self.df['white_win'] = (self.df['white_score'] == 1.0).astype(int)
        
        if 'black_win' not in self.df.columns:
            self.df['black_win'] = (self.df['white_score'] == 0.0).astype(int)
        
        # Catégories
        if 'elo_category' not in self.df.columns:
            self.df['elo_category'] = pd.cut(self.df['avg_elo'], 
                                           bins=[0, 1200, 1500, 1800, 2100, 3000],
                                           labels=['Débutant', 'Intermédiaire', 'Avancé', 'Expert', 'Maître'])
        
        if 'duration_category' not in self.df.columns:
            self.df['duration_category'] = pd.cut(self.df['moves_count'],
                                               bins=[0, 15, 30, 50, 200],
                                               labels=['Très courte', 'Courte', 'Moyenne', 'Longue'])
        
        print("✅ Données préparées pour l'analyse")
    
    def filter_data(self, elo_range=None, time_control=None, color=None, 
                   moves_range=None, min_games=10):
        """Filtre les données selon les critères"""
        
        filtered_df = self.df.copy()
        
        # Filtre ELO
        if elo_range:
            min_elo, max_elo = elo_range
            filtered_df = filtered_df[
                (filtered_df['avg_elo'] >= min_elo) & 
                (filtered_df['avg_elo'] <= max_elo)
            ]
        
        # Filtre cadence
        if time_control:
            if isinstance(time_control, str):
                time_control = [time_control]
            filtered_df = filtered_df[filtered_df['time_control'].isin(time_control)]
        
        # Filtre couleur (blanc=1, noir=0)
        if color is not None:
            if color == 'white':
                # Analyser les performances des blancs
                pass  # Pas de filtre spécial nécessaire
            elif color == 'black':
                # Inverser les scores pour analyser du point de vue noir
                filtered_df = filtered_df.copy()
                filtered_df['white_score'] = 1 - filtered_df['white_score']
                filtered_df['black_score'] = 1 - filtered_df['black_score']
        
        # Filtre nombre de coups
        if moves_range:
            min_moves, max_moves = moves_range
            filtered_df = filtered_df[
                (filtered_df['moves_count'] >= min_moves) & 
                (filtered_df['moves_count'] <= max_moves)
            ]
        
        # Filtre minimum de parties par ouverture
        if min_games > 0:
            opening_counts = filtered_df['main_opening'].value_counts()
            valid_openings = opening_counts[opening_counts >= min_games].index
            filtered_df = filtered_df[filtered_df['main_opening'].isin(valid_openings)]
        
        return filtered_df

# Créer l'analyseur
analyzer = OpeningAnalyzer(df)
print("🔧 Analyseur initialisé et prêt")


✅ Données préparées pour l'analyse
🔧 Analyseur initialisé et prêt


In [18]:
def analyze_global_stats(analyzer):
    """Analyse globale de la base de données"""
    
    print("🔎 ANALYSE GLOBALE")
    print("=" * 40)
    
    df = analyzer.df
    
    # Statistiques générales
    print(f"📊 Total parties: {len(df):,}")
    print(f"📚 Ouvertures uniques: {df['main_opening'].nunique():,}")
    print(f"🎯 ELO moyen: {df['avg_elo'].mean():.0f}")
    print(f"📈 ELO médian: {df['avg_elo'].median():.0f}")
    print(f"♟️ Coups moyen: {df['moves_count'].mean():.1f}")
    
    # Distribution des résultats
    white_winrate = df['white_score'].mean()
    draw_rate = (df['white_score'] == 0.5).mean()
    black_winrate = (df['white_score'] == 0.0).mean()
    
    print(f"\n🏆 DISTRIBUTION DES RÉSULTATS:")
    print(f"   Victoires blancs: {white_winrate:.1%}")
    print(f"   Nulles: {draw_rate:.1%}")
    print(f"   Victoires noirs: {black_winrate:.1%}")
    
    # Top ouvertures les plus jouées
    print(f"\n📈 TOP 10 OUVERTURES LES PLUS JOUÉES:")
    top_openings = df['main_opening'].value_counts().head(10)
    for i, (opening, count) in enumerate(top_openings.items(), 1):
        pct = count / len(df) * 100
        print(f"   {i:2d}. {opening:<30} {count:>5,} ({pct:4.1f}%)")
    
    # Ouvertures rares (potentiel surprise)
    print(f"\n🎯 OUVERTURES RARES (< 1% des parties):")
    rare_openings = df['main_opening'].value_counts()
    rare_threshold = len(df) * 0.01
    rare_openings = rare_openings[rare_openings < rare_threshold].tail(10)
    
    for opening, count in rare_openings.items():
        pct = count / len(df) * 100
        print(f"   • {opening:<30} {count:>3,} ({pct:4.2f}%)")
    
    # Distribution par cadence
    print(f"\n⏱️ DISTRIBUTION PAR CADENCE:")
    time_dist = df['time_control'].value_counts()
    for tc, count in time_dist.items():
        pct = count / len(df) * 100
        print(f"   {tc:<12} {count:>5,} ({pct:4.1f}%)")
    
    # Distribution par niveau ELO
    if 'elo_category' in df.columns:
        print(f"\n🎯 DISTRIBUTION PAR NIVEAU:")
        elo_dist = df['elo_category'].value_counts().sort_index()
        for category, count in elo_dist.items():
            pct = count / len(df) * 100
            print(f"   {category:<15} {count:>5,} ({pct:4.1f}%)")

# Exécuter l'analyse globale
analyze_global_stats(analyzer)

🔎 ANALYSE GLOBALE
📊 Total parties: 49,360
📚 Ouvertures uniques: 192
🎯 ELO moyen: 1611
📈 ELO médian: 1606
♟️ Coups moyen: 68.3

🏆 DISTRIBUTION DES RÉSULTATS:
   Victoires blancs: 52.3%
   Nulles: 3.2%
   Victoires noirs: 46.1%

📈 TOP 10 OUVERTURES LES PLUS JOUÉES:
    1. Sicilian Defense               4,969 (10.1%)
    2. French Defense                 3,538 ( 7.2%)
    3. Queen's Pawn Game              2,516 ( 5.1%)
    4. King's Pawn Game               2,250 ( 4.6%)
    5. Scandinavian Defense           2,195 ( 4.4%)
    6. Italian Game                   1,550 ( 3.1%)
    7. English Opening                1,545 ( 3.1%)
    8. Bishop's Opening               1,481 ( 3.0%)
    9. Caro-Kann Defense              1,388 ( 2.8%)
   10. Ruy Lopez                      1,189 ( 2.4%)

🎯 OUVERTURES RARES (< 1% des parties):
   • King's Gambit, Falkbeer Countergambit, Nimzowitsch-Marshall Countergambit   5 (0.01%)
   • King's Gambit Declined, Norwalde Variation   5 (0.01%)
   • King's Gambit Accept

In [19]:
def analyze_popularity(analyzer):
    """Analyse détaillée de la popularité des ouvertures"""
    
    print("\n📈 ANALYSE DE POPULARITÉ")
    print("=" * 40)
    
    df = analyzer.df
    
    # 1. Popularité globale (déjà fait dans analyse globale)
    
    # 2. Popularité par tranche ELO
    print("🎯 POPULARITÉ PAR NIVEAU ELO:")
    
    elo_ranges = [
        ('Débutant (< 1200)', 0, 1200),
        ('Intermédiaire (1200-1500)', 1200, 1500),
        ('Avancé (1500-1800)', 1500, 1800),
        ('Expert (1800-2100)', 1800, 2100),
        ('Maître (> 2100)', 2100, 3000)
    ]
    
    for name, min_elo, max_elo in elo_ranges:
        filtered_df = analyzer.filter_data(elo_range=(min_elo, max_elo), min_games=5)
        if len(filtered_df) > 0:
            print(f"\n   {name} ({len(filtered_df):,} parties):")
            top_5 = filtered_df['main_opening'].value_counts().head(5)
            for i, (opening, count) in enumerate(top_5.items(), 1):
                pct = count / len(filtered_df) * 100
                print(f"      {i}. {opening:<25} {count:>3,} ({pct:4.1f}%)")
    
    # 3. Popularité par cadence
    print(f"\n⏱️ POPULARITÉ PAR CADENCE:")
    
    for time_control in df['time_control'].unique():
        filtered_df = analyzer.filter_data(time_control=time_control, min_games=5)
        if len(filtered_df) > 0:
            print(f"\n   {time_control} ({len(filtered_df):,} parties):")
            top_5 = filtered_df['main_opening'].value_counts().head(5)
            for i, (opening, count) in enumerate(top_5.items(), 1):
                pct = count / len(filtered_df) * 100
                print(f"      {i}. {opening:<25} {count:>3,} ({pct:4.1f}%)")
    
    # 4. Ouvertures "oubliées" (faible fréquence mais haut niveau)
    print(f"\n🔍 OUVERTURES 'OUBLIÉES' (Rares mais jouées à haut niveau):")
    
    high_elo_df = analyzer.filter_data(elo_range=(1800, 3000), min_games=3)
    if len(high_elo_df) > 0:
        high_elo_openings = high_elo_df['main_opening'].value_counts()
        
        # Comparer avec la popularité globale
        global_openings = df['main_opening'].value_counts()
        
        forgotten_openings = []
        for opening in high_elo_openings.index:
            high_elo_freq = high_elo_openings[opening] / len(high_elo_df)
            global_freq = global_openings[opening] / len(df)
            
            # Ouvertures plus populaires à haut niveau qu'en général
            if high_elo_freq > global_freq * 1.5 and global_freq < 0.05:
                forgotten_openings.append((opening, high_elo_openings[opening], 
                                          high_elo_freq, global_freq))
        
        forgotten_openings.sort(key=lambda x: x[2], reverse=True)
        
        for opening, count, high_freq, global_freq in forgotten_openings[:10]:
            print(f"   • {opening:<30} {count:>3} parties ({high_freq:5.1%} vs {global_freq:5.1%} global)")

# Exécuter l'analyse de popularité
analyze_popularity(analyzer)


📈 ANALYSE DE POPULARITÉ
🎯 POPULARITÉ PAR NIVEAU ELO:

   Débutant (< 1200) (548 parties):
      1. King's Pawn Game          104 (19.0%)
      2. Queen's Pawn Game          38 ( 6.9%)
      3. French Defense             35 ( 6.4%)
      4. Scandinavian Defense       33 ( 6.0%)
      5. Bishop's Opening           30 ( 5.5%)

   Intermédiaire (1200-1500) (13,575 parties):
      1. King's Pawn Game          1,192 ( 8.8%)
      2. French Defense            880 ( 6.5%)
      3. Sicilian Defense          851 ( 6.3%)
      4. Queen's Pawn Game         851 ( 6.3%)
      5. Scandinavian Defense      649 ( 4.8%)

   Avancé (1500-1800) (26,847 parties):
      1. Sicilian Defense          3,051 (11.4%)
      2. French Defense            2,034 ( 7.6%)
      3. Queen's Pawn Game         1,211 ( 4.5%)
      4. Scandinavian Defense      1,113 ( 4.1%)
      5. Italian Game              916 ( 3.4%)

   Expert (1800-2100) (7,748 parties):
      1. Sicilian Defense          996 (12.9%)
      2. French De

In [20]:
def analyze_performance(analyzer):
    """Analyse des performances (winrates) des ouvertures"""
    
    print("\n🧠 ANALYSE DE PERFORMANCE (WINRATE)")
    print("=" * 45)
    
    df = analyzer.df
    
    def calculate_opening_stats(filtered_df, min_games=10):
        """Calcule les statistiques par ouverture"""
        
        stats = []
        
        for opening in filtered_df['main_opening'].unique():
            opening_df = filtered_df[filtered_df['main_opening'] == opening]
            
            if len(opening_df) >= min_games:
                stats.append({
                    'opening': opening,
                    'games': len(opening_df),
                    'white_winrate': opening_df['white_score'].mean(),
                    'draw_rate': (opening_df['white_score'] == 0.5).mean(),
                    'black_winrate': (opening_df['white_score'] == 0.0).mean(),
                    'avg_moves': opening_df['moves_count'].mean(),
                    'avg_elo': opening_df['avg_elo'].mean()
                })
        
        return pd.DataFrame(stats)
    
    # 1. Winrate global
    print("🏆 MEILLEURES OUVERTURES (Winrate Blancs Global):")
    global_stats = calculate_opening_stats(df, min_games=20)
    
    if len(global_stats) > 0:
        best_global = global_stats.nlargest(10, 'white_winrate')
        for i, row in enumerate(best_global.itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.white_winrate:5.1%} "
                  f"({row.games:>3,} parties, {row.avg_elo:4.0f} ELO)")
    
    # 2. Winrate à bas ELO
    print(f"\n🎯 MEILLEURES OUVERTURES BAS ELO (< 1200):")
    low_elo_df = analyzer.filter_data(elo_range=(0, 1200), min_games=10)
    low_elo_stats = calculate_opening_stats(low_elo_df, min_games=10)
    
    if len(low_elo_stats) > 0:
        best_low_elo = low_elo_stats.nlargest(8, 'white_winrate')
        for i, row in enumerate(best_low_elo.itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.white_winrate:5.1%} "
                  f"({row.games:>3,} parties)")
    
    # 3. Winrate à haut ELO
    print(f"\n🏅 MEILLEURES OUVERTURES HAUT ELO (> 1800):")
    high_elo_df = analyzer.filter_data(elo_range=(1800, 3000), min_games=5)
    high_elo_stats = calculate_opening_stats(high_elo_df, min_games=5)
    
    if len(high_elo_stats) > 0:
        best_high_elo = high_elo_stats.nlargest(8, 'white_winrate')
        for i, row in enumerate(best_high_elo.itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.white_winrate:5.1%} "
                  f"({row.games:>3,} parties)")
    
    # 4. Winrate par cadence
    print(f"\n⏱️ MEILLEURES OUVERTURES PAR CADENCE:")
    
    for time_control in ['Bullet', 'Blitz', 'Rapid']:
        tc_df = analyzer.filter_data(time_control=time_control, min_games=10)
        tc_stats = calculate_opening_stats(tc_df, min_games=10)
        
        if len(tc_stats) > 0:
            print(f"\n   {time_control}:")
            best_tc = tc_stats.nlargest(5, 'white_winrate')
            for i, row in enumerate(best_tc.itertuples(), 1):
                print(f"      {i}. {row.opening:<25} {row.white_winrate:5.1%} "
                      f"({row.games:>3,} parties)")
    
    # 5. Ouvertures équilibrées (bon winrate + volume)
    print(f"\n⚖️ OUVERTURES FIABLES (Bon winrate + Volume suffisant):")
    if len(global_stats) > 0:
        # Filtrer: winrate > 52% ET au moins 50 parties
        reliable = global_stats[
            (global_stats['white_winrate'] > 0.52) & 
            (global_stats['games'] >= 50)
        ].sort_values('white_winrate', ascending=False)
        
        for i, row in enumerate(reliable.head(10).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.white_winrate:5.1%} "
                  f"({row.games:>3,} parties)")
    
    # 6. Piéges potentiels (bon winrate + peu jouées)
    print(f"\n🎣 PIÈGES POTENTIELS (Bon winrate + Rares):")
    if len(global_stats) > 0:
        # Filtrer: winrate > 55% ET moins de 2% des parties
        total_games = len(df)
        traps = global_stats[
            (global_stats['white_winrate'] > 0.55) & 
            (global_stats['games'] < total_games * 0.02) &
            (global_stats['games'] >= 10)
        ].sort_values('white_winrate', ascending=False)
        
        for i, row in enumerate(traps.head(8).itertuples(), 1):
            pct = row.games / total_games * 100
            print(f"   {i:2d}. {row.opening:<30} {row.white_winrate:5.1%} "
                  f"({row.games:>3,} parties, {pct:4.1f}%)")

# Exécuter l'analyse de performance
analyze_performance(analyzer)


🧠 ANALYSE DE PERFORMANCE (WINRATE)
🏆 MEILLEURES OUVERTURES (Winrate Blancs Global):
    1. Barnes Defense                 82.1% ( 39 parties, 1498 ELO)
    2. Carr Defense                   80.0% ( 25 parties, 1620 ELO)
    3. Queen's Gambit                 76.4% ( 53 parties, 1567 ELO)
    4. Slav Defense #3                75.9% ( 29 parties, 1651 ELO)
    5. Pirc Defense #2                70.8% ( 36 parties, 1586 ELO)
    6. King's Gambit Accepted, MacLeod Defense 69.3% ( 83 parties, 1600 ELO)
    7. Danish Gambit Accepted, Classical Defense 68.2% ( 22 parties, 1550 ELO)
    8. Ware Defense                   67.9% ( 39 parties, 1426 ELO)
    9. Duras Gambit                   67.1% ( 41 parties, 1591 ELO)
   10. King's Gambit Accepted, Cunningham Defense 66.3% ( 49 parties, 1658 ELO)

🎯 MEILLEURES OUVERTURES BAS ELO (< 1200):
    1. Alekhine Defense               71.4% ( 14 parties)
    2. Queen's Pawn Game              61.8% ( 38 parties)
    3. Four Knights Game              61.5% 

In [21]:
def analyze_duration(analyzer):
    """Analyse de la durée des parties par ouverture"""
    
    print("\n⏱️ ANALYSE DE DURÉE (NOMBRE DE COUPS)")
    print("=" * 42)
    
    df = analyzer.df
    
    def calculate_duration_stats(filtered_df, min_games=10):
        """Calcule les statistiques de durée par ouverture"""
        
        stats = []
        
        for opening in filtered_df['main_opening'].unique():
            opening_df = filtered_df[filtered_df['main_opening'] == opening]
            
            if len(opening_df) >= min_games:
                stats.append({
                    'opening': opening,
                    'games': len(opening_df),
                    'avg_moves': opening_df['moves_count'].mean(),
                    'median_moves': opening_df['moves_count'].median(),
                    'white_winrate': opening_df['white_score'].mean(),
                    'short_games_pct': (opening_df['moves_count'] <= 20).mean(),
                    'long_games_pct': (opening_df['moves_count'] >= 50).mean()
                })
        
        return pd.DataFrame(stats)
    
    # 1. Ouvertures donnant les parties les plus courtes
    print("⚡ OUVERTURES LES PLUS COURTES:")
    duration_stats = calculate_duration_stats(df, min_games=15)
    
    if len(duration_stats) > 0:
        shortest = duration_stats.nsmallest(10, 'avg_moves')
        for i, row in enumerate(shortest.itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.avg_moves:4.1f} coups "
                  f"({row.games:>3,} parties, WR: {row.white_winrate:4.1%})")
    
    # 2. Ouvertures donnant les parties les plus longues
    print(f"\n🐌 OUVERTURES LES PLUS LONGUES:")
    if len(duration_stats) > 0:
        longest = duration_stats.nlargest(8, 'avg_moves')
        for i, row in enumerate(longest.itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.avg_moves:4.1f} coups "
                  f"({row.games:>3,} parties, WR: {row.white_winrate:4.1%})")
    
    # 3. Ouvertures courtes ET gagnantes
    print(f"\n🚀 OUVERTURES COURTES ET GAGNANTES:")
    if len(duration_stats) > 0:
        short_winners = duration_stats[
            (duration_stats['avg_moves'] < 35) & 
            (duration_stats['white_winrate'] > 0.53)
        ].sort_values('white_winrate', ascending=False)
        
        for i, row in enumerate(short_winners.head(8).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.avg_moves:4.1f} coups, "
                  f"WR: {row.white_winrate:4.1%} ({row.games:>3,} parties)")
    
    # 4. Victoires rapides fréquentes (pièges d'ouverture)
    print(f"\n🎯 PIÈGES D'OUVERTURE (Victoires très rapides):")
    if len(duration_stats) > 0:
        traps = duration_stats[
            (duration_stats['short_games_pct'] > 0.15) &  # > 15% de parties < 20 coups
            (duration_stats['white_winrate'] > 0.55) &    # Bon winrate
            (duration_stats['games'] >= 10)               # Volume minimal
        ].sort_values('short_games_pct', ascending=False)
        
        for i, row in enumerate(traps.head(6).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} {row.short_games_pct:4.1%} rapides, "
                  f"WR: {row.white_winrate:4.1%} ({row.games:>3,} parties)")
    
    # 5. Relation ELO et durée
    print(f"\n📊 ÉVOLUTION DE LA DURÉE PAR NIVEAU ELO:")
    
    elo_ranges = [
        ('Débutant', 0, 1200),
        ('Intermédiaire', 1200, 1500),
        ('Avancé', 1500, 1800),
        ('Expert', 1800, 2100),
        ('Maître', 2100, 3000)
    ]
    
    for name, min_elo, max_elo in elo_ranges:
        elo_df = analyzer.filter_data(elo_range=(min_elo, max_elo), min_games=20)
        if len(elo_df) > 0:
            avg_duration = elo_df['moves_count'].mean()
            print(f"   {name:<15} {avg_duration:4.1f} coups en moyenne ({len(elo_df):,} parties)")

# Exécuter l'analyse de durée
analyze_duration(analyzer)


⏱️ ANALYSE DE DURÉE (NOMBRE DE COUPS)
⚡ OUVERTURES LES PLUS COURTES:
    1. Danish Gambit Accepted, Copenhagen Defense 50.9 coups ( 18 parties, WR: 77.8%)
    2. King's Gambit Declined         51.4 coups ( 32 parties, WR: 60.9%)
    3. Petrov                         51.6 coups ( 17 parties, WR: 64.7%)
    4. Borg Defense                   51.8 coups ( 46 parties, WR: 64.1%)
    5. Blackmar-Diemer Gambit         54.8 coups ( 54 parties, WR: 50.0%)
    6. King's Gambit Accepted, Bishop's Gambit 55.6 coups ( 18 parties, WR: 63.9%)
    7. Goldsmith Defense              55.6 coups ( 22 parties, WR: 63.6%)
    8. Danish Gambit Accepted, Classical Defense 56.0 coups ( 22 parties, WR: 68.2%)
    9. King's Gambit Accepted, Modern Defense 56.5 coups ( 17 parties, WR: 52.9%)
   10. King's Gambit Accepted, Becker Defense 56.8 coups ( 33 parties, WR: 65.2%)

🐌 OUVERTURES LES PLUS LONGUES:
    1. Latvian Gambit Accepted        84.7 coups ( 23 parties, WR: 50.0%)
    2. Kangaroo Defense             

In [22]:
def analyze_by_time_control(analyzer):
    """Analyse spécialisée par cadence de jeu"""
    
    print("\n🕰️ ANALYSE PAR CADENCE")
    print("=" * 30)
    
    df = analyzer.df
    
    def opening_performance_by_tc(opening, time_controls):
        """Compare les performances d'une ouverture selon la cadence"""
        
        results = {}
        for tc in time_controls:
            tc_df = analyzer.filter_data(time_control=tc)
            opening_df = tc_df[tc_df['main_opening'] == opening]
            
            if len(opening_df) >= 5:
                results[tc] = {
                    'games': len(opening_df),
                    'winrate': opening_df['white_score'].mean(),
                    'avg_moves': opening_df['moves_count'].mean()
                }
        
        return results
    
    time_controls = ['Bullet', 'Blitz', 'Rapid', 'Classical']
    
    # 1. Analyse détaillée par cadence
    for tc in time_controls:
        tc_df = analyzer.filter_data(time_control=tc, min_games=10)
        if len(tc_df) == 0:
            continue
            
        print(f"\n🎯 ANALYSE {tc.upper()}:")
        print(f"   Total parties: {len(tc_df):,}")
        print(f"   Durée moyenne: {tc_df['moves_count'].mean():.1f} coups")
        print(f"   Winrate blancs: {tc_df['white_score'].mean():.1%}")
        
        # Top ouvertures pour cette cadence
        tc_stats = []
        for opening in tc_df['main_opening'].unique():
            opening_df = tc_df[tc_df['main_opening'] == opening]
            if len(opening_df) >= 10:
                tc_stats.append({
                    'opening': opening,
                    'games': len(opening_df),
                    'winrate': opening_df['white_score'].mean(),
                    'avg_moves': opening_df['moves_count'].mean()
                })
        
        if tc_stats:
            tc_stats_df = pd.DataFrame(tc_stats)
            best_tc = tc_stats_df.nlargest(5, 'winrate')
            
            print(f"   🏆 Top 5 ouvertures:")
            for i, row in enumerate(best_tc.itertuples(), 1):
                print(f"      {i}. {row.opening:<25} {row.winrate:5.1%} "
                      f"({row.games:>3,} parties, {row.avg_moves:4.1f} coups)")
    
    # 2. Ouvertures avec variations importantes selon la cadence
    print(f"\n🔄 OUVERTURES AVEC VARIATIONS SELON LA CADENCE:")
    
    # Analyser les ouvertures populaires dans plusieurs cadences
    popular_openings = df['main_opening'].value_counts().head(15).index
    
    cadence_variations = []
    
    for opening in popular_openings:
        performances = opening_performance_by_tc(opening, time_controls)
        
        if len(performances) >= 3:  # Au moins 3 cadences
            winrates = [p['winrate'] for p in performances.values()]
            if len(winrates) >= 2:
                variation = max(winrates) - min(winrates)
                if variation > 0.08:  # Variation > 8%
                    cadence_variations.append((opening, variation, performances))
    
    cadence_variations.sort(key=lambda x: x[1], reverse=True)
    
    for opening, variation, performances in cadence_variations[:8]:
        print(f"\n   📊 {opening}:")
        print(f"      Variation: {variation:.1%}")
        for tc, stats in performances.items():
            print(f"      {tc:<10} {stats['winrate']:5.1%} ({stats['games']:>3,} parties)")
    
    # 3. Spécialités par cadence
    print(f"\n⚡ SPÉCIALITÉS PAR CADENCE:")
    
    specialties = {
        'Bullet': [],
        'Blitz': [],
        'Rapid': [],
        'Classical': []
    }
    
    for tc in time_controls:
        tc_df = analyzer.filter_data(time_control=tc, min_games=15)
        if len(tc_df) == 0:
            continue
        
        # Calculer les performances relatives
        for opening in tc_df['main_opening'].unique():
            opening_df = tc_df[tc_df['main_opening'] == opening]
            if len(opening_df) >= 15:
                tc_winrate = opening_df['white_score'].mean()
                
                # Comparer avec la performance globale de cette ouverture
                global_opening_df = df[df['main_opening'] == opening]
                global_winrate = global_opening_df['white_score'].mean()
                
                # Si significativement meilleure dans cette cadence
                if tc_winrate > global_winrate + 0.05 and tc_winrate > 0.53:
                    specialties[tc].append((opening, tc_winrate, len(opening_df)))
    
    for tc, specialty_list in specialties.items():
        if specialty_list:
            specialty_list.sort(key=lambda x: x[1], reverse=True)
            print(f"\n   {tc}:")
            for opening, winrate, games in specialty_list[:5]:
                print(f"      • {opening:<25} {winrate:5.1%} ({games:>3,} parties)")

# Exécuter l'analyse par cadence
analyze_by_time_control(analyzer)


🕰️ ANALYSE PAR CADENCE

🎯 ANALYSE BULLET:
   Total parties: 14,698
   Durée moyenne: 69.6 coups
   Winrate blancs: 52.3%
   🏆 Top 5 ouvertures:
      1. Reti Opening #2           86.4% ( 11 parties, 72.5 coups)
      2. Slav Defense #3           83.3% ( 12 parties, 87.2 coups)
      3. King's Gambit Accepted, MacLeod Defense 83.3% ( 24 parties, 51.1 coups)
      4. King's Gambit Accepted, Schallopp Defense 80.0% ( 10 parties, 54.7 coups)
      5. King's Gambit Accepted, King's Knight Gambit 76.5% ( 17 parties, 48.2 coups)

🎯 ANALYSE BLITZ:
   Total parties: 26,257
   Durée moyenne: 68.3 coups
   Winrate blancs: 52.5%
   🏆 Top 5 ouvertures:
      1. Queen's Gambit            85.7% ( 21 parties, 59.7 coups)
      2. Barnes Defense            85.7% ( 14 parties, 48.8 coups)
      3. Carr Defense              83.3% ( 18 parties, 70.8 coups)
      4. Duras Gambit              82.6% ( 23 parties, 62.0 coups)
      5. Colle System #2           81.8% ( 11 parties, 69.4 coups)

🎯 ANALYSE RAPID

In [23]:
def analyze_useful_combinations(analyzer):
    """Analyse des combinaisons utiles pour l'utilisateur"""
    
    print("\n🎯 COMBINAISONS UTILES POUR L'UTILISATEUR")
    print("=" * 45)
    
    df = analyzer.df
    
    def calculate_comprehensive_stats(filtered_df, min_games=10):
        """Calcule des statistiques complètes par ouverture"""
        
        stats = []
        
        for opening in filtered_df['main_opening'].unique():
            opening_df = filtered_df[filtered_df['main_opening'] == opening]
            
            if len(opening_df) >= min_games:
                stats.append({
                    'opening': opening,
                    'games': len(opening_df),
                    'winrate': opening_df['white_score'].mean(),
                    'draw_rate': (opening_df['white_score'] == 0.5).mean(),
                    'avg_moves': opening_df['moves_count'].mean(),
                    'avg_elo': opening_df['avg_elo'].mean(),
                    'short_games_pct': (opening_df['moves_count'] <= 20).mean(),
                    'popularity_pct': len(opening_df) / len(filtered_df) * 100
                })
        
        return pd.DataFrame(stats)
    
    # 1. Ouvertures efficaces pour débutants
    print("🎓 OUVERTURES RECOMMANDÉES POUR DÉBUTANTS:")
    print("   (Bon winrate + Jouées à bas ELO + Pas trop complexes)")
    
    beginner_df = analyzer.filter_data(elo_range=(800, 1400), min_games=10)
    beginner_stats = calculate_comprehensive_stats(beginner_df, min_games=15)
    
    if len(beginner_stats) > 0:
        # Critères: winrate > 52%, durée raisonnable, popularité > 2%
        beginner_picks = beginner_stats[
            (beginner_stats['winrate'] > 0.52) & 
            (beginner_stats['avg_moves'] < 45) &
            (beginner_stats['popularity_pct'] > 2)
        ].sort_values('winrate', ascending=False)
        
        for i, row in enumerate(beginner_picks.head(8).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} WR: {row.winrate:5.1%}, "
                  f"Coups: {row.avg_moves:4.1f}, Pop: {row.popularity_pct:4.1f}%")
    
    # 2. Victoires rapides pour débutants
    print(f"\n⚡ VICTOIRES RAPIDES POUR DÉBUTANTS:")
    print("   (Courtes + Gagnantes + Bas ELO)")
    
    if len(beginner_stats) > 0:
        quick_wins = beginner_stats[
            (beginner_stats['winrate'] > 0.55) & 
            (beginner_stats['avg_moves'] < 30) &
            (beginner_stats['games'] >= 10)
        ].sort_values(['winrate', 'short_games_pct'], ascending=[False, False])
        
        for i, row in enumerate(quick_wins.head(6).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} WR: {row.winrate:5.1%}, "
                  f"Coups: {row.avg_moves:4.1f} ({row.short_games_pct:4.1%} rapides)")
    
    # 3. Pièges efficaces (bon winrate + peu jouées)
    print(f"\n🎣 PIÈGES EFFICACES:")
    print("   (Excellent winrate + Rares + Volume suffisant)")
    
    global_stats = calculate_comprehensive_stats(df, min_games=15)
    
    if len(global_stats) > 0:
        traps = global_stats[
            (global_stats['winrate'] > 0.58) & 
            (global_stats['popularity_pct'] < 3) &
            (global_stats['games'] >= 15)
        ].sort_values('winrate', ascending=False)
        
        for i, row in enumerate(traps.head(8).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} WR: {row.winrate:5.1%}, "
                  f"Pop: {row.popularity_pct:4.1f}% ({row.games:>3,} parties)")
    
    # 4. Ouvertures solides (beaucoup de nulles)
    print(f"\n🛡️ OUVERTURES SOLIDES:")
    print("   (Beaucoup de nulles + Winrate décent)")
    
    if len(global_stats) > 0:
        solid = global_stats[
            (global_stats['draw_rate'] > 0.35) & 
            (global_stats['winrate'] > 0.45) &
            (global_stats['winrate'] < 0.55) &
            (global_stats['games'] >= 20)
        ].sort_values('draw_rate', ascending=False)
        
        for i, row in enumerate(solid.head(6).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} Nulles: {row.draw_rate:5.1%}, "
                  f"WR: {row.winrate:5.1%} ({row.games:>3,} parties)")
    
    # 5. Pièges à éviter
    print(f"\n⚠️ PIÈGES À ÉVITER:")
    print("   (Populaires mais mauvais winrate)")
    
    if len(global_stats) > 0:
        avoid = global_stats[
            (global_stats['popularity_pct'] > 5) & 
            (global_stats['winrate'] < 0.47)
        ].sort_values('winrate', ascending=True)
        
        for i, row in enumerate(avoid.head(5).itertuples(), 1):
            print(f"   {i:2d}. {row.opening:<30} WR: {row.winrate:5.1%}, "
                  f"Pop: {row.popularity_pct:4.1f}% - ⚠️ ÉVITER")
    
    # 6. Spécialités par cadence avec critères stricts
    print(f"\n🎯 SPÉCIALITÉS PAR CADENCE (Recommandations):")
    
    for tc in ['Bullet', 'Blitz', 'Rapid']:
        tc_df = analyzer.filter_data(time_control=tc, min_games=15)
        tc_stats = calculate_comprehensive_stats(tc_df, min_games=15)
        
        if len(tc_stats) > 0:
            tc_picks = tc_stats[
                (tc_stats['winrate'] > 0.54) & 
                (tc_stats['games'] >= 15)
            ].sort_values('winrate', ascending=False)
            
            if len(tc_picks) > 0:
                print(f"\n   {tc}:")
                for i, row in enumerate(tc_picks.head(4).itertuples(), 1):
                    print(f"      {i}. {row.opening:<25} {row.winrate:5.1%} "
                          f"({row.games:>3,} parties)")

# Exécuter l'analyse des combinaisons utiles
analyze_useful_combinations(analyzer)


🎯 COMBINAISONS UTILES POUR L'UTILISATEUR
🎓 OUVERTURES RECOMMANDÉES POUR DÉBUTANTS:
   (Bon winrate + Jouées à bas ELO + Pas trop complexes)

⚡ VICTOIRES RAPIDES POUR DÉBUTANTS:
   (Courtes + Gagnantes + Bas ELO)

🎣 PIÈGES EFFICACES:
   (Excellent winrate + Rares + Volume suffisant)
    1. Barnes Defense                 WR: 82.1%, Pop:  0.1% ( 39 parties)
    2. Carr Defense                   WR: 80.0%, Pop:  0.1% ( 25 parties)
    3. Danish Gambit Accepted, Copenhagen Defense WR: 77.8%, Pop:  0.0% ( 18 parties)
    4. Queen's Gambit                 WR: 76.4%, Pop:  0.1% ( 53 parties)
    5. Slav Defense #3                WR: 75.9%, Pop:  0.1% ( 29 parties)
    6. Canard Opening                 WR: 73.3%, Pop:  0.0% ( 15 parties)
    7. Pirc Defense #2                WR: 70.8%, Pop:  0.1% ( 36 parties)
    8. King's Gambit Accepted, MacLeod Defense WR: 69.3%, Pop:  0.2% ( 83 parties)

🛡️ OUVERTURES SOLIDES:
   (Beaucoup de nulles + Winrate décent)

⚠️ PIÈGES À ÉVITER:
   (Populaires ma

In [24]:
def analyze_advanced_insights(analyzer):
    """Analyses diverses et insights avancés"""
    
    print("\n🔀 ANALYSES DIVERSES ET INSIGHTS AVANCÉS")
    print("=" * 45)
    
    df = analyzer.df
    
    # 1. Ouvertures avec fort déséquilibre selon la couleur
    print("⚖️ OUVERTURES AVEC DÉSÉQUILIBRE DE COULEUR:")
    print("   (Performances très différentes selon qui joue)")
    
    color_imbalances = []
    
    for opening in df['main_opening'].value_counts().head(20).index:
        opening_df = df[df['main_opening'] == opening]
        
        if len(opening_df) >= 20:
            # Performance des blancs (score normal)
            white_performance = opening_df['white_score'].mean()
            
            # Performance des noirs (1 - score)
            black_performance = 1 - white_performance
            
            imbalance = abs(white_performance - black_performance)
            
            if imbalance > 0.15:  # Plus de 15% de différence
                color_imbalances.append((
                    opening, 
                    white_performance, 
                    black_performance, 
                    imbalance,
                    len(opening_df)
                ))
    
    color_imbalances.sort(key=lambda x: x[3], reverse=True)
    
    for opening, white_perf, black_perf, imbalance, games in color_imbalances[:8]:
        better_color = "Blancs" if white_perf > black_perf else "Noirs"
        print(f"   • {opening:<30} {better_color} +{imbalance:4.1%} "
              f"(B:{white_perf:4.1%} vs N:{black_perf:4.1%}, {games} parties)")
    
    # 2. Ouvertures les plus symétriques
    print(f"\n🔄 OUVERTURES LES PLUS SYMÉTRIQUES:")
    print("   (Résultats équilibrés ~50/50)")
    
    symmetric_openings = []
    
    for opening in df['main_opening'].value_counts().head(25).index:
        opening_df = df[df['main_opening'] == opening]
        
        if len(opening_df) >= 30:
            white_winrate = opening_df['white_score'].mean()
            symmetry_score = 1 - abs(white_winrate - 0.5)  # Plus proche de 0.5 = plus symétrique
            
            if 0.48 <= white_winrate <= 0.52:  # Entre 48% et 52%
                symmetric_openings.append((
                    opening,
                    white_winrate,
                    symmetry_score,
                    len(opening_df)
                ))
    
    symmetric_openings.sort(key=lambda x: x[2], reverse=True)
    
    for opening, winrate, symmetry, games in symmetric_openings[:6]:
        print(f"   • {opening:<30} {winrate:5.1%} "
              f"(Symétrie: {symmetry:5.1%}, {games:>3,} parties)")
    
    # 3. Évolution des performances selon le niveau
    print(f"\n📈 ÉVOLUTION SELON LE NIVEAU ELO:")
    print("   (Ouvertures qui changent d'efficacité)")
    
    elo_evolution = []
    
    # Analyser quelques ouvertures populaires
    popular_openings = df['main_opening'].value_counts().head(12).index
    
    for opening in popular_openings:
        opening_df = df[df['main_opening'] == opening]
        
        if len(opening_df) >= 50:
            # Performance à bas ELO (< 1400)
            low_elo_df = opening_df[opening_df['avg_elo'] < 1400]
            # Performance à haut ELO (> 1600)
            high_elo_df = opening_df[opening_df['avg_elo'] > 1600]
            
            if len(low_elo_df) >= 15 and len(high_elo_df) >= 15:
                low_winrate = low_elo_df['white_score'].mean()
                high_winrate = high_elo_df['white_score'].mean()
                evolution = high_winrate - low_winrate
                
                if abs(evolution) > 0.05:  # Changement > 5%
                    elo_evolution.append((
                        opening,
                        low_winrate,
                        high_winrate,
                        evolution,
                        len(low_elo_df),
                        len(high_elo_df)
                    ))
    
    elo_evolution.sort(key=lambda x: abs(x[3]), reverse=True)
    
    for opening, low_wr, high_wr, evolution, low_games, high_games in elo_evolution[:8]:
        trend = "📈 Améliore" if evolution > 0 else "📉 Détériore"
        print(f"   • {opening:<30} {trend} {abs(evolution):4.1%} "
              f"({low_wr:4.1%}→{high_wr:4.1%})")
    
    # 4. Familles d'ouvertures
    print(f"\n🏠 PERFORMANCE PAR FAMILLE D'OUVERTURES:")
    
    # Grouper par famille (première partie du nom)
    df['opening_family'] = df['main_opening'].str.split().str[0]
    
    family_stats = []
    for family in df['opening_family'].value_counts().head(10).index:
        family_df = df[df['opening_family'] == family]
        
        if len(family_df) >= 30:
            family_stats.append({
                'family': family,
                'games': len(family_df),
                'openings': family_df['main_opening'].nunique(),
                'winrate': family_df['white_score'].mean(),
                'avg_moves': family_df['moves_count'].mean(),
                'popularity': len(family_df) / len(df) * 100
            })
    
    family_stats_df = pd.DataFrame(family_stats)
    family_stats_df = family_stats_df.sort_values('winrate', ascending=False)
    
    for i, row in enumerate(family_stats_df.head(8).itertuples(), 1):
        print(f"   {i:2d}. {row.family:<20} {row.winrate:5.1%} "
              f"({row.games:>4,} parties, {row.openings:>2} ouvertures)")
    
    # 5. Ouvertures "gagne-vite"
    print(f"\n🚀 OUVERTURES 'GAGNE-VITE':")
    print("   (Excellent winrate + Très courtes)")
    
    quick_wins = []
    
    for opening in df['main_opening'].value_counts().index:
        opening_df = df[df['main_opening'] == opening]
        
        if len(opening_df) >= 15:
            winrate = opening_df['white_score'].mean()
            avg_moves = opening_df['moves_count'].mean()
            quick_games_pct = (opening_df['moves_count'] <= 25).mean()
            
            # Score composite: winrate pondéré par la rapidité
            if winrate > 0.55 and avg_moves < 35:
                speed_score = winrate * (1 + quick_games_pct)
                quick_wins.append((
                    opening,
                    winrate,
                    avg_moves,
                    quick_games_pct,
                    speed_score,
                    len(opening_df)
                ))
    
    quick_wins.sort(key=lambda x: x[4], reverse=True)
    
    for opening, winrate, avg_moves, quick_pct, score, games in quick_wins[:8]:
        print(f"   • {opening:<30} WR:{winrate:5.1%}, "
              f"Coups:{avg_moves:4.1f}, Rapides:{quick_pct:4.1%} ({games} parties)")

# Exécuter les analyses avancées
analyze_advanced_insights(analyzer)


🔀 ANALYSES DIVERSES ET INSIGHTS AVANCÉS
⚖️ OUVERTURES AVEC DÉSÉQUILIBRE DE COULEUR:
   (Performances très différentes selon qui joue)
   • Philidor Defense #3            Blancs +18.5% (B:59.2% vs N:40.8%, 692 parties)
   • Philidor Defense               Blancs +18.2% (B:59.1% vs N:40.9%, 730 parties)

🔄 OUVERTURES LES PLUS SYMÉTRIQUES:
   (Résultats équilibrés ~50/50)
   • Scandinavian Defense           50.0% (Symétrie: 100.0%, 2,195 parties)
   • French Defense                 50.8% (Symétrie: 99.2%, 3,538 parties)
   • Zukertort Opening              51.0% (Symétrie: 99.0%, 673 parties)
   • Caro-Kann Defense              48.7% (Symétrie: 98.7%, 1,388 parties)
   • King's Pawn Game               51.8% (Symétrie: 98.2%, 2,250 parties)

📈 ÉVOLUTION SELON LE NIVEAU ELO:
   (Ouvertures qui changent d'efficacité)
   • Bishop's Opening               📉 Détériore 14.8% (64.1%→49.2%)
   • Scandinavian Defense           📉 Détériore 12.2% (57.5%→45.3%)
   • Italian Game                   📉 Dété

In [25]:
def create_analysis_visualizations(analyzer):
    """Crée des visualisations pour l'analyse des ouvertures"""
    
    print("\n🎨 GÉNÉRATION DES VISUALISATIONS")
    print("=" * 35)
    
    df = analyzer.df
    
    # Préparer les données pour les graphiques
    opening_stats = []
    
    for opening in df['main_opening'].value_counts().head(15).index:
        opening_df = df[df['main_opening'] == opening]
        
        opening_stats.append({
            'opening': opening,
            'games': len(opening_df),
            'winrate': opening_df['white_score'].mean(),
            'avg_moves': opening_df['moves_count'].mean(),
            'avg_elo': opening_df['avg_elo'].mean(),
            'popularity': len(opening_df) / len(df) * 100
        })
    
    stats_df = pd.DataFrame(opening_stats)
    
    try:
        # 1. Graphique Popularité vs Performance
        fig1 = px.scatter(
            stats_df, 
            x='popularity', 
            y='winrate',
            size='games',
            hover_data=['opening', 'avg_moves', 'avg_elo'],
            title="📊 Popularité vs Performance des Ouvertures",
            labels={
                'popularity': 'Popularité (%)',
                'winrate': 'Winrate Blancs',
                'games': 'Nombre de parties'
            }
        )
        
        fig1.add_hline(y=0.5, line_dash="dash", line_color="gray", 
                      annotation_text="Équilibre (50%)")
        
        # Sauvegarder
        os.makedirs('../data/exported', exist_ok=True)
        html_file1 = f"../data/exported/popularity_vs_performance_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
        fig1.write_html(html_file1)
        print(f"✅ Graphique 1 sauvegardé: {html_file1}")
        
        # 2. Graphique Durée vs Performance
        fig2 = px.scatter(
            stats_df,
            x='avg_moves',
            y='winrate', 
            size='games',
            color='avg_elo',
            hover_data=['opening', 'popularity'],
            title="⏱️ Durée vs Performance des Ouvertures",
            labels={
                'avg_moves': 'Durée moyenne (coups)',
                'winrate': 'Winrate Blancs',
                'avg_elo': 'ELO moyen'
            }
        )
        
        html_file2 = f"../data/exported/duration_vs_performance_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
        fig2.write_html(html_file2)
        print(f"✅ Graphique 2 sauvegardé: {html_file2}")
        
        # 3. Analyse par niveau ELO
        elo_analysis = []
        elo_ranges = [
            ('Débutant', 0, 1200),
            ('Intermédiaire', 1200, 1500), 
            ('Avancé', 1500, 1800),
            ('Expert', 1800, 3000)
        ]
        
        for level, min_elo, max_elo in elo_ranges:
            level_df = analyzer.filter_data(elo_range=(min_elo, max_elo), min_games=10)
            
            for opening in level_df['main_opening'].value_counts().head(8).index:
                opening_df = level_df[level_df['main_opening'] == opening]
                if len(opening_df) >= 10:
                    elo_analysis.append({
                        'niveau': level,
                        'ouverture': opening,
                        'parties': len(opening_df),
                        'winrate': opening_df['white_score'].mean(),
                        'popularite': len(opening_df) / len(level_df) * 100
                    })
        
        if elo_analysis:
            elo_df = pd.DataFrame(elo_analysis)
            
            fig3 = px.bar(
                elo_df,
                x='ouverture',
                y='winrate',
                color='niveau',
                title="🎯 Performance des Ouvertures par Niveau ELO",
                labels={
                    'ouverture': 'Ouverture',
                    'winrate': 'Winrate Blancs',
                    'niveau': 'Niveau ELO'
                }
            )
            
            fig3.update_xaxes(tickangle=45)
            
            html_file3 = f"../data/exported/performance_by_elo_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
            fig3.write_html(html_file3)
            print(f"✅ Graphique 3 sauvegardé: {html_file3}")
        
        # 4. Dashboard de synthèse
        fig4 = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Top Ouvertures (Popularité)', 'Top Ouvertures (Performance)',
                           'Distribution Durée', 'Performance par Cadence'),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                   [{"type": "histogram"}, {"type": "bar"}]]
        )
        
        # Top popularité
        top_popular = stats_df.nlargest(8, 'popularity')
        fig4.add_trace(
            go.Bar(x=top_popular['opening'], y=top_popular['popularity'],
                   name='Popularité', marker_color='lightblue'),
            row=1, col=1
        )
        
        # Top performance
        top_performance = stats_df.nlargest(8, 'winrate')
        fig4.add_trace(
            go.Bar(x=top_performance['opening'], y=top_performance['winrate'],
                   name='Winrate', marker_color='lightgreen'),
            row=1, col=2
        )
        
        # Distribution durée
        fig4.add_trace(
            go.Histogram(x=df['moves_count'], name='Durée parties',
                        marker_color='orange', nbinsx=30),
            row=2, col=1
        )
        
        # Performance par cadence
        tc_performance = []
        for tc in df['time_control'].unique():
            tc_df = analyzer.filter_data(time_control=tc)
            if len(tc_df) > 50:
                tc_performance.append({
                    'cadence': tc,
                    'winrate': tc_df['white_score'].mean()
                })
        
        if tc_performance:
            tc_df = pd.DataFrame(tc_performance)
            fig4.add_trace(
                go.Bar(x=tc_df['cadence'], y=tc_df['winrate'],
                       name='Winrate par cadence', marker_color='purple'),
                row=2, col=2
            )
        
        fig4.update_layout(
            title_text="📈 Dashboard d'Analyse des Ouvertures",
            height=800,
            showlegend=False
        )
        
        # Rotation des labels
        fig4.update_xaxes(tickangle=45, row=1, col=1)
        fig4.update_xaxes(tickangle=45, row=1, col=2)
        
        html_file4 = f"../data/exported/dashboard_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
        fig4.write_html(html_file4)
        print(f"✅ Dashboard sauvegardé: {html_file4}")
        
        print(f"\n🌐 Pour voir les visualisations, ouvrez les fichiers HTML dans votre navigateur")
        
        # Tentative d'affichage dans le notebook (peut échouer)
        try:
            fig1.show()
            print("✅ Graphiques également affichés dans le notebook!")
        except:
            print("💡 Utilisez les fichiers HTML pour voir les graphiques interactifs")
            
    except Exception as e:
        print(f"⚠️ Erreur lors de la création des visualisations: {e}")
        print("📊 L'analyse textuelle reste disponible ci-dessus")

# Exécuter la génération de visualisations
create_analysis_visualizations(analyzer)


🎨 GÉNÉRATION DES VISUALISATIONS
✅ Graphique 1 sauvegardé: ../data/exported/popularity_vs_performance_20250607_233732.html
✅ Graphique 2 sauvegardé: ../data/exported/duration_vs_performance_20250607_233732.html
✅ Graphique 3 sauvegardé: ../data/exported/performance_by_elo_20250607_233732.html
✅ Dashboard sauvegardé: ../data/exported/dashboard_analysis_20250607_233732.html

🌐 Pour voir les visualisations, ouvrez les fichiers HTML dans votre navigateur
✅ Graphiques également affichés dans le notebook!


In [26]:
def generate_custom_reports(analyzer):
    """Génère des rapports personnalisés selon les besoins"""
    
    print("\n📋 GÉNÉRATEUR DE RAPPORTS PERSONNALISÉS")
    print("=" * 45)
    
    df = analyzer.df
    
    def generate_beginner_report():
        """Rapport spécialisé pour débutants"""
        
        report = []
        report.append("🎓 RAPPORT POUR DÉBUTANTS")
        report.append("=" * 30)
        report.append("")
        
        # Filtrer pour débutants
        beginner_df = analyzer.filter_data(elo_range=(800, 1400), min_games=10)
        
        if len(beginner_df) > 0:
            report.append(f"📊 Analyse basée sur {len(beginner_df):,} parties de niveau débutant (800-1400 ELO)")
            report.append("")
            
            # Top 5 recommandations
            stats = []
            for opening in beginner_df['main_opening'].unique():
                opening_df = beginner_df[beginner_df['main_opening'] == opening]
                if len(opening_df) >= 15:
                    stats.append({
                        'opening': opening,
                        'winrate': opening_df['white_score'].mean(),
                        'games': len(opening_df),
                        'avg_moves': opening_df['moves_count'].mean(),
                        'popularity': len(opening_df) / len(beginner_df) * 100
                    })
            
            if stats:
                stats_df = pd.DataFrame(stats)
                
                # Recommandations équilibrées
                recommendations = stats_df[
                    (stats_df['winrate'] > 0.52) & 
                    (stats_df['popularity'] > 3) &
                    (stats_df['avg_moves'] < 50)
                ].sort_values('winrate', ascending=False)
                
                report.append("🎯 TOP 5 OUVERTURES RECOMMANDÉES:")
                for i, row in enumerate(recommendations.head(5).itertuples(), 1):
                    report.append(f"   {i}. {row.opening}")
                    report.append(f"      • Winrate: {row.winrate:.1%}")
                    report.append(f"      • Popularité: {row.popularity:.1f}% des parties")
                    report.append(f"      • Durée moyenne: {row.avg_moves:.1f} coups")
                    report.append(f"      • Basé sur {row.games:,} parties")
                    report.append("")
                
                # Pièges à éviter
                avoid = stats_df[
                    (stats_df['winrate'] < 0.47) & 
                    (stats_df['popularity'] > 5)
                ].sort_values('winrate', ascending=True)
                
                if len(avoid) > 0:
                    report.append("⚠️ OUVERTURES À ÉVITER:")
                    for i, row in enumerate(avoid.head(3).itertuples(), 1):
                        report.append(f"   {i}. {row.opening} - Winrate: {row.winrate:.1%}")
                    report.append("")
        
        return "\n".join(report)
    
    def generate_tactical_report():
        """Rapport pour joueurs cherchant des pièges tactiques"""
        
        report = []
        report.append("🎣 RAPPORT TACTIQUE - PIÈGES ET SURPRISES")
        report.append("=" * 45)
        report.append("")
        
        # Analyser tous les niveaux
        stats = []
        for opening in df['main_opening'].unique():
            opening_df = df[df['main_opening'] == opening]
            if len(opening_df) >= 10:
                stats.append({
                    'opening': opening,
                    'winrate': opening_df['white_score'].mean(),
                    'games': len(opening_df),
                    'avg_moves': opening_df['moves_count'].mean(),
                    'popularity': len(opening_df) / len(df) * 100,
                    'quick_wins': (opening_df['moves_count'] <= 25).mean()
                })
        
        if stats:
            stats_df = pd.DataFrame(stats)
            
            # Pièges efficaces
            traps = stats_df[
                (stats_df['winrate'] > 0.58) & 
                (stats_df['popularity'] < 4) &
                (stats_df['games'] >= 10)
            ].sort_values('winrate', ascending=False)
            
            report.append("🎯 PIÈGES EFFICACES (Bon winrate + Rares):")
            for i, row in enumerate(traps.head(8).itertuples(), 1):
                report.append(f"   {i}. {row.opening}")
                report.append(f"      • Winrate: {row.winrate:.1%}")
                report.append(f"      • Rareté: {row.popularity:.1f}% (effet surprise)")
                report.append(f"      • Parties rapides: {row.quick_wins:.1%}")
                report.append(f"      • Volume: {row.games:,} parties")
                report.append("")
            
            # Victoires express
            quick_traps = stats_df[
                (stats_df['winrate'] > 0.55) & 
                (stats_df['avg_moves'] < 30) &
                (stats_df['quick_wins'] > 0.2)
            ].sort_values(['winrate', 'quick_wins'], ascending=[False, False])
            
            report.append("⚡ VICTOIRES EXPRESS:")
            for i, row in enumerate(quick_traps.head(6).itertuples(), 1):
                report.append(f"   {i}. {row.opening}")
                report.append(f"      • Winrate: {row.winrate:.1%}")
                report.append(f"      • Durée: {row.avg_moves:.1f} coups")
                report.append(f"      • Victoires rapides: {row.quick_wins:.1%}")
                report.append("")
        
        return "\n".join(report)
    
    def generate_competitive_report():
        """Rapport pour joueurs compétitifs"""
        
        report = []
        report.append("🏆 RAPPORT COMPÉTITIF - HAUT NIVEAU")
        report.append("=" * 40)
        report.append("")
        
        # Analyser haut niveau
        competitive_df = analyzer.filter_data(elo_range=(1600, 3000), min_games=10)
        
        if len(competitive_df) > 0:
            report.append(f"📊 Analyse basée sur {len(competitive_df):,} parties de niveau compétitif (1600+ ELO)")
            report.append("")
            
            stats = []
            for opening in competitive_df['main_opening'].unique():
                opening_df = competitive_df[competitive_df['main_opening'] == opening]
                if len(opening_df) >= 15:
                    stats.append({
                        'opening': opening,
                        'winrate': opening_df['white_score'].mean(),
                        'games': len(opening_df),
                        'avg_moves': opening_df['moves_count'].mean(),
                        'draw_rate': (opening_df['white_score'] == 0.5).mean(),
                        'avg_elo': opening_df['avg_elo'].mean()
                    })
            
            if stats:
                stats_df = pd.DataFrame(stats)
                
                # Ouvertures fiables
                reliable = stats_df[
                    (stats_df['winrate'] > 0.51) & 
                    (stats_df['games'] >= 30)
                ].sort_values('winrate', ascending=False)
                
                report.append("🎯 OUVERTURES FIABLES (Haut niveau):")
                for i, row in enumerate(reliable.head(8).itertuples(), 1):
                    report.append(f"   {i}. {row.opening}")
                    report.append(f"      • Winrate: {row.winrate:.1%}")
                    report.append(f"      • ELO moyen: {row.avg_elo:.0f}")
                    report.append(f"      • Durée: {row.avg_moves:.1f} coups")
                    report.append(f"      • Nulles: {row.draw_rate:.1%}")
                    report.append(f"      • Volume: {row.games:,} parties")
                    report.append("")
                
                # Ouvertures solides
                solid = stats_df[
                    (stats_df['draw_rate'] > 0.35) & 
                    (stats_df['winrate'] > 0.48) &
                    (stats_df['winrate'] < 0.52)
                ].sort_values('draw_rate', ascending=False)
                
                report.append("🛡️ OUVERTURES SOLIDES (Égalité):")
                for i, row in enumerate(solid.head(5).itertuples(), 1):
                    report.append(f"   {i}. {row.opening} - Nulles: {row.draw_rate:.1%}, WR: {row.winrate:.1%}")
                report.append("")
        
        return "\n".join(report)
    
    # Générer les rapports
    print("📝 Génération des rapports...")
    
    beginner_report = generate_beginner_report()
    tactical_report = generate_tactical_report()
    competitive_report = generate_competitive_report()
    
    # Sauvegarder les rapports
    os.makedirs('../data/exported', exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Rapport débutant
    with open(f'../data/exported/rapport_debutant_{timestamp}.txt', 'w', encoding='utf-8') as f:
        f.write(beginner_report)
    print(f"✅ Rapport débutant sauvegardé")
    
    # Rapport tactique
    with open(f'../data/exported/rapport_tactique_{timestamp}.txt', 'w', encoding='utf-8') as f:
        f.write(tactical_report)
    print(f"✅ Rapport tactique sauvegardé")
    
    # Rapport compétitif
    with open(f'../data/exported/rapport_competitif_{timestamp}.txt', 'w', encoding='utf-8') as f:
        f.write(competitive_report)
    print(f"✅ Rapport compétitif sauvegardé")
    
    # Afficher un aperçu
    print(f"\n📋 APERÇU DU RAPPORT DÉBUTANT:")
    print("-" * 40)
    print(beginner_report[:800] + "..." if len(beginner_report) > 800 else beginner_report)

# Exécuter la génération de rapports
generate_custom_reports(analyzer)


📋 GÉNÉRATEUR DE RAPPORTS PERSONNALISÉS
📝 Génération des rapports...
✅ Rapport débutant sauvegardé
✅ Rapport tactique sauvegardé
✅ Rapport compétitif sauvegardé

📋 APERÇU DU RAPPORT DÉBUTANT:
----------------------------------------
🎓 RAPPORT POUR DÉBUTANTS

📊 Analyse basée sur 6,382 parties de niveau débutant (800-1400 ELO)

🎯 TOP 5 OUVERTURES RECOMMANDÉES:


In [27]:
def create_query_interface(analyzer):
    """Interface pour des requêtes personnalisées"""
    
    print("\n🔍 INTERFACE DE REQUÊTES PERSONNALISÉES")
    print("=" * 45)
    
    df = analyzer.df
    
    def custom_query(elo_min=None, elo_max=None, time_control=None, 
                    min_winrate=None, max_popularity=None, min_games=10,
                    max_moves=None, min_moves=None):
        """Requête personnalisée avec filtres multiples"""
        
        # Appliquer les filtres
        filtered_df = df.copy()
        
        if elo_min is not None or elo_max is not None:
            elo_min = elo_min or 0
            elo_max = elo_max or 3000
            filtered_df = analyzer.filter_data(elo_range=(elo_min, elo_max), min_games=0)
        
        if time_control:
            filtered_df = analyzer.filter_data(time_control=time_control, min_games=0)
        
        if min_moves is not None or max_moves is not None:
            moves_min = min_moves or 0
            moves_max = max_moves or 200
            filtered_df = filtered_df[
                (filtered_df['moves_count'] >= moves_min) & 
                (filtered_df['moves_count'] <= moves_max)
            ]
        
        # Calculer les statistiques
        results = []
        for opening in filtered_df['main_opening'].unique():
            opening_df = filtered_df[filtered_df['main_opening'] == opening]
            
            if len(opening_df) >= min_games:
                winrate = opening_df['white_score'].mean()
                popularity = len(opening_df) / len(filtered_df) * 100
                
                # Appliquer les filtres de performance
                if min_winrate and winrate < min_winrate:
                    continue
                if max_popularity and popularity > max_popularity:
                    continue
                
                results.append({
                    'opening': opening,
                    'games': len(opening_df),
                    'winrate': winrate,
                    'popularity': popularity,
                    'avg_moves': opening_df['moves_count'].mean(),
                    'avg_elo': opening_df['avg_elo'].mean(),
                    'draw_rate': (opening_df['white_score'] == 0.5).mean()
                })
        
        return pd.DataFrame(results).sort_values('winrate', ascending=False)
    
    # Exemples de requêtes prédéfinies
    print("🎯 EXEMPLES DE REQUÊTES PRÉDÉFINIES:")
    print()
    
    # 1. Ouvertures pour débutants agressifs
    print("1. 🔥 OUVERTURES POUR DÉBUTANTS AGRESSIFS:")
    print("   (ELO < 1400, Winrate > 55%, Parties courtes)")
    
    aggressive_beginners = custom_query(
        elo_max=1400,
        min_winrate=0.55,
        max_moves=35,
        min_games=15
    )
    
    if len(aggressive_beginners) > 0:
        for i, row in enumerate(aggressive_beginners.head(5).itertuples(), 1):
            print(f"      {i}. {row.opening:<25} WR:{row.winrate:5.1%} "
                  f"Coups:{row.avg_moves:4.1f} ({row.games:>3,} parties)")
    else:
        print("      Aucun résultat trouvé")
    
    print()
    
    # 2. Pièges secrets
    print("2. 🎭 PIÈGES SECRETS:")
    print("   (Winrate > 60%, Popularité < 2%, Volume suffisant)")
    
    secret_traps = custom_query(
        min_winrate=0.60,
        max_popularity=2,
        min_games=8
    )
    
    if len(secret_traps) > 0:
        for i, row in enumerate(secret_traps.head(5).itertuples(), 1):
            print(f"      {i}. {row.opening:<25} WR:{row.winrate:5.1%} "
                  f"Pop:{row.popularity:4.1f}% ({row.games:>2,} parties)")
    else:
        print("      Aucun résultat trouvé")
    
    print()
    
    # 3. Ouvertures équilibrées pour compétition
    print("3. ⚖️ OUVERTURES ÉQUILIBRÉES POUR COMPÉTITION:")
    print("   (ELO > 1600, Winrate 48-52%, Beaucoup de nulles)")
    
    balanced_competitive = df[df['avg_elo'] > 1600]
    balanced_stats = []
    
    for opening in balanced_competitive['main_opening'].unique():
        opening_df = balanced_competitive[balanced_competitive['main_opening'] == opening]
        if len(opening_df) >= 20:
            winrate = opening_df['white_score'].mean()
            draw_rate = (opening_df['white_score'] == 0.5).mean()
            
            if 0.48 <= winrate <= 0.52 and draw_rate > 0.30:
                balanced_stats.append({
                    'opening': opening,
                    'winrate': winrate,
                    'draw_rate': draw_rate,
                    'games': len(opening_df)
                })
    
    if balanced_stats:
        balanced_df = pd.DataFrame(balanced_stats).sort_values('draw_rate', ascending=False)
        for i, row in enumerate(balanced_df.head(5).itertuples(), 1):
            print(f"      {i}. {row.opening:<25} WR:{row.winrate:5.1%} "
                  f"Nulles:{row.draw_rate:5.1%} ({row.games:>3,} parties)")
    else:
        print("      Aucun résultat trouvé")
    
    print()
    
    # 4. Spécialités Bullet
    print("4. ⚡ SPÉCIALITÉS BULLET:")
    print("   (Cadence Bullet, Winrate > 54%, Populaires)")
    
    bullet_specials = custom_query(
        time_control='Bullet',
        min_winrate=0.54,
        min_games=20
    )
    
    if len(bullet_specials) > 0:
        for i, row in enumerate(bullet_specials.head(5).itertuples(), 1):
            print(f"      {i}. {row.opening:<25} WR:{row.winrate:5.1%} "
                  f"Pop:{row.popularity:4.1f}% ({row.games:>3,} parties)")
    else:
        print("      Aucun résultat trouvé")
    
    # Guide d'utilisation
    print(f"\n💡 GUIDE D'UTILISATION DES REQUÊTES:")
    print("=" * 40)
    print("""
Pour créer vos propres requêtes, utilisez la fonction custom_query() avec :

Paramètres disponibles:
• elo_min, elo_max : Plage d'ELO (ex: 1200, 1800)
• time_control : 'Bullet', 'Blitz', 'Rapid', 'Classical'
• min_winrate : Winrate minimum (ex: 0.55 pour 55%)
• max_popularity : Popularité maximum (ex: 3 pour 3%)
• min_games : Nombre minimum de parties (ex: 20)
• min_moves, max_moves : Plage de nombre de coups

Exemples d'utilisation:
# Ouvertures rares mais efficaces à haut niveau
results = custom_query(elo_min=1800, min_winrate=0.57, max_popularity=1.5)

# Parties rapides en blitz
results = custom_query(time_control='Blitz', max_moves=25, min_winrate=0.53)

# Ouvertures solides pour débutants
results = custom_query(elo_max=1300, min_winrate=0.50, min_games=30)
    """)

# Exécuter l'interface de requêtes
create_query_interface(analyzer)


🔍 INTERFACE DE REQUÊTES PERSONNALISÉES
🎯 EXEMPLES DE REQUÊTES PRÉDÉFINIES:

1. 🔥 OUVERTURES POUR DÉBUTANTS AGRESSIFS:
   (ELO < 1400, Winrate > 55%, Parties courtes)
      1. Ruy Lopez                 WR:81.8% Coups:25.0 ( 22 parties)
      2. Queen's Pawn Game #2      WR:78.8% Coups:26.3 ( 33 parties)
      3. Bishop's Opening          WR:73.8% Coups:19.4 ( 84 parties)
      4. Queen's Gambit Refused    WR:73.3% Coups:21.5 ( 15 parties)
      5. Russian Game              WR:72.2% Coups:21.4 ( 36 parties)

2. 🎭 PIÈGES SECRETS:
   (Winrate > 60%, Popularité < 2%, Volume suffisant)
      1. King's Gambit, Falkbeer Countergambit Accepted WR:82.1% Pop: 0.0% (14 parties)
      2. Barnes Defense            WR:82.1% Pop: 0.1% (39 parties)
      3. Carr Defense              WR:80.0% Pop: 0.1% (25 parties)
      4. Slav Indian               WR:77.8% Pop: 0.0% ( 9 parties)
      5. Danish Gambit Accepted, Copenhagen Defense WR:77.8% Pop: 0.0% (18 parties)

3. ⚖️ OUVERTURES ÉQUILIBRÉES POUR COMP

In [28]:
def generate_final_summary(analyzer):
    """Génère un résumé final avec recommandations personnalisées"""
    
    print("\n🎯 RÉSUMÉ FINAL ET RECOMMANDATIONS")
    print("=" * 45)
    
    df = analyzer.df
    
    # Statistiques générales
    total_games = len(df)
    total_openings = df['main_opening'].nunique()
    avg_elo = df['avg_elo'].mean()
    
    print(f"📊 STATISTIQUES DE L'ANALYSE:")
    print(f"   • {total_games:,} parties analysées")
    print(f"   • {total_openings:,} ouvertures uniques")
    print(f"   • ELO moyen: {avg_elo:.0f}")
    print(f"   • Durée moyenne: {df['moves_count'].mean():.1f} coups")
    
    # Top découvertes
    print(f"\n🏆 PRINCIPALES DÉCOUVERTES:")
    
    # Meilleure ouverture globale
    opening_stats = []
    for opening in df['main_opening'].value_counts().head(20).index:
        opening_df = df[df['main_opening'] == opening]
        if len(opening_df) >= 50:  # Volume significatif
            opening_stats.append({
                'opening': opening,
                'winrate': opening_df['white_score'].mean(),
                'games': len(opening_df),
                'popularity': len(opening_df) / total_games * 100
            })
    
    if opening_stats:
        best_opening = max(opening_stats, key=lambda x: x['winrate'])
        most_popular = max(opening_stats, key=lambda x: x['popularity'])
        
        print(f"   🥇 Meilleure ouverture (volume significatif):")
        print(f"      {best_opening['opening']} - {best_opening['winrate']:.1%} winrate")
        print(f"   📈 Ouverture la plus populaire:")
        print(f"      {most_popular['opening']} - {most_popular['popularity']:.1f}% des parties")
    
    # Recommandations par profil
    print(f"\n🎯 RECOMMANDATIONS PAR PROFIL DE JOUEUR:")
    
    # Profil débutant
    print(f"\n   🎓 POUR LES DÉBUTANTS (< 1400 ELO):")
    beginner_df = analyzer.filter_data(elo_range=(800, 1400), min_games=15)
    
    if len(beginner_df) > 0:
        beginner_recs = []
        for opening in beginner_df['main_opening'].unique():
            opening_df = beginner_df[beginner_df['main_opening'] == opening]
            if len(opening_df) >= 15:
                winrate = opening_df['white_score'].mean()
                popularity = len(opening_df) / len(beginner_df) * 100
                
                if winrate > 0.52 and popularity > 3:
                    beginner_recs.append((opening, winrate, popularity))
        
        beginner_recs.sort(key=lambda x: x[1], reverse=True)
        
        for i, (opening, winrate, pop) in enumerate(beginner_recs[:3], 1):
            print(f"      {i}. {opening} - {winrate:.1%} winrate, {pop:.1f}% popularité")
    
    # Profil intermédiaire
    print(f"\n   📚 POUR LES JOUEURS INTERMÉDIAIRES (1400-1700 ELO):")
    intermediate_df = analyzer.filter_data(elo_range=(1400, 1700), min_games=15)
    
    if len(intermediate_df) > 0:
        intermediate_recs = []
        for opening in intermediate_df['main_opening'].unique():
            opening_df = intermediate_df[intermediate_df['main_opening'] == opening]
            if len(opening_df) >= 15:
                winrate = opening_df['white_score'].mean()
                avg_moves = opening_df['moves_count'].mean()
                
                if winrate > 0.51 and 25 < avg_moves < 50:  # Équilibré
                    intermediate_recs.append((opening, winrate, avg_moves))
        
        intermediate_recs.sort(key=lambda x: x[1], reverse=True)
        
        for i, (opening, winrate, moves) in enumerate(intermediate_recs[:3], 1):
            print(f"      {i}. {opening} - {winrate:.1%} winrate, {moves:.1f} coups")
    
    # Profil avancé
    print(f"\n   🏅 POUR LES JOUEURS AVANCÉS (> 1700 ELO):")
    advanced_df = analyzer.filter_data(elo_range=(1700, 3000), min_games=10)
    
    if len(advanced_df) > 0:
        advanced_recs = []
        for opening in advanced_df['main_opening'].unique():
            opening_df = advanced_df[advanced_df['main_opening'] == opening]
            if len(opening_df) >= 10:
                winrate = opening_df['white_score'].mean()
                draw_rate = (opening_df['white_score'] == 0.5).mean()
                
                if winrate > 0.50:  # Critères plus stricts
                    advanced_recs.append((opening, winrate, draw_rate, len(opening_df)))
        
        advanced_recs.sort(key=lambda x: x[1], reverse=True)
        
        for i, (opening, winrate, draws, games) in enumerate(advanced_recs[:3], 1):
            print(f"      {i}. {opening} - {winrate:.1%} winrate, {draws:.1%} nulles ({games} parties)")
    
    # Conseils stratégiques
    print(f"\n💡 CONSEILS STRATÉGIQUES:")
    print(f"   • Privilégiez les ouvertures avec un bon équilibre winrate/popularité")
    print(f"   • Évitez les ouvertures trop populaires avec un mauvais winrate")
    print(f"   • Adaptez votre répertoire selon la cadence de jeu")
    print(f"   • Gardez quelques 'pièges' rares pour surprendre")
    print(f"   • Analysez vos propres parties pour personnaliser ces recommandations")
    
    # Fichiers générés
    print(f"\n📁 FICHIERS GÉNÉRÉS:")
    print(f"   • Rapports détaillés: /data/exported/rapport_*.txt")
    print(f"   • Visualisations: /data/exported/*.html")
    print(f"   • Données nettoyées: /data/processed/")
    
    print(f"\n✅ ANALYSE TERMINÉE - Bonne chance pour vos prochaines parties ! ♟️")

# Exécuter le résumé final
generate_final_summary(analyzer)


🎯 RÉSUMÉ FINAL ET RECOMMANDATIONS
📊 STATISTIQUES DE L'ANALYSE:
   • 49,360 parties analysées
   • 192 ouvertures uniques
   • ELO moyen: 1611
   • Durée moyenne: 68.3 coups

🏆 PRINCIPALES DÉCOUVERTES:
   🥇 Meilleure ouverture (volume significatif):
      Philidor Defense #3 - 59.2% winrate
   📈 Ouverture la plus populaire:
      Sicilian Defense - 10.1% des parties

🎯 RECOMMANDATIONS PAR PROFIL DE JOUEUR:

   🎓 POUR LES DÉBUTANTS (< 1400 ELO):
      1. Bishop's Opening - 63.8% winrate, 4.1% popularité
      2. Scandinavian Defense - 57.7% winrate, 6.2% popularité
      3. French Defense - 56.6% winrate, 6.1% popularité

   📚 POUR LES JOUEURS INTERMÉDIAIRES (1400-1700 ELO):
      1. King's Gambit Declined - 68.2% winrate, 49.1 coups

   🏅 POUR LES JOUEURS AVANCÉS (> 1700 ELO):
      1. King's Gambit Accepted, King's Knight Gambit - 82.7% winrate, 3.8% nulles (26 parties)
      2. Slav Defense #3 - 81.8% winrate, 0.0% nulles (11 parties)
      3. Danish Gambit Accepted - 81.8% winrate, 